In [1]:
!pip install transformers[onnx]
!pip install onnxruntime


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
model_name = 'microsoft/MiniLM-L12-H384-uncased'

In [3]:
!python -m transformers.onnx --model=data/output/best_model --feature=sequence-classification onnx/

2023-04-06 14:10:27.568965: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'cudart64_110.dll'; dlerror: cudart64_110.dll not found
2023-04-06 14:10:27.569015: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
C:\anaconda\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
Local PyTorch model found.
Framework not requested. Using torch to export to ONNX.
Using framework PyTorch: 1.13.1+cpu
Overriding 1 configuration item(s)
	- use_cache -> False
Validating ONNX model...
	-[\u2713] ONNX model output names match reference model ({'logits'})
	- Validating ONNX Model output "logits":
		-[\u2713] (3, 3) matches (3, 3)
		-[\u2713] all values close (atol: 1e-05)
All good, model saved at: onnx/m

In [4]:
import pandas as pd
test_data = pd.read_csv('data/input/test.csv')

C:\anaconda\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [5]:
test_tweets = test_data['tweet'].to_list()
test_labels = test_data['class'].to_list()

In [6]:
import numpy as np
from transformers import AutoTokenizer
from onnxruntime import InferenceSession

tokenizer = AutoTokenizer.from_pretrained(model_name)
session = InferenceSession("onnx/model.onnx")
# ONNX Runtime expects NumPy arrays as input
inputs = tokenizer(test_tweets, return_tensors="np", max_length=64, padding="max_length", truncation=True)
# convert input data type to int64
inputs = {k: v.astype(np.int64) for k, v in inputs.items()}
outputs = session.run(output_names=["logits"], input_feed=inputs)

In [7]:
test_output_labels_1 = np.argmax(outputs[0], axis=-1)

In [8]:
from sklearn.metrics import f1_score

f1_model_1 = f1_score(test_labels, test_output_labels_1, average='weighted')
f1_model_1

0.9026632384728706

## Same Model with Quantization

In [9]:
from onnxruntime.quantization import quantize_dynamic, QuantType
quantize_dynamic("onnx/model.onnx", "onnx/model_int8.onnx", weight_type=QuantType.QUInt8)

Ignore MatMul due to non constant B: /[/bert/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.5/atten

In [10]:
session_int = InferenceSession("onnx/model_int8.onnx")
outputs_int = session_int.run(output_names=["logits"], input_feed=inputs)

In [11]:
test_output_labels_2 = np.argmax(outputs_int[0], axis=-1)

In [12]:
f1_model_2 = f1_score(test_labels, test_output_labels_2, average='weighted')
f1_model_2

0.9022992897211402

The quantization has no significant effect on the model F1 score while reducing the size from 127MB to 32.2MB.